<a href="https://colab.research.google.com/github/Shantanu-Meta/Light-Pollution-Research/blob/main/LSTM_forecast_statewise_2025%2C2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

def forecast_urban_sol(state_data, time_steps=5, epochs=100, batch_size=32):
  """
  Forecasts Urban_SOL for a given state using LSTM.

  Args:
    state_data: DataFrame containing data for the specific state.
    time_steps: Number of time steps for input sequence.
    epochs: Number of epochs for model training.
    batch_size: Batch size for model training.

  Returns:
    A tuple containing:
      - Forecasted Urban_SOL values for the next 5 years.
      - RMSE, MAE, and R-squared scores.
  """

  # Preprocess data
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
  urban_sol = state_data['Urban_SOL'].dropna().values.reshape(-1, 1)

  # Normalize data
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(urban_sol)

  # Create dataset for LSTM
  X, Y = [], []
  for i in range(len(scaled_data) - time_steps - 1):
    a = scaled_data[i:(i + time_steps), 0]
    X.append(a)
    Y.append(scaled_data[i + time_steps, 0])
  X, Y = np.array(X), np.array(Y)
  X = X.reshape(X.shape[0], X.shape[1], 1)

  # Split into train and test sets
  train_size = int(len(X) * 0.8)
  X_train, X_test = X[:train_size], X[train_size:]
  Y_train, Y_test = Y[:train_size], Y[train_size:]

  # Build the LSTM model
  model = Sequential([
      LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
      Dropout(0.2),
      LSTM(units=50),
      Dropout(0.2),
      Dense(units=1)
  ])

  model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

  # Predict and inverse transform
  predictions = model.predict(X_test)
  predictions = scaler.inverse_transform(predictions)
  Y_test_actual = scaler.inverse_transform(Y_test.reshape(-1, 1))

  # Evaluate model performance
  rmse = np.sqrt(mean_squared_error(Y_test_actual, predictions))
  mae = mean_absolute_error(Y_test_actual, predictions)
  r2 = r2_score(Y_test_actual, predictions)

  # Forecast future values
  future_years = 5
  last_sequence = scaled_data[-time_steps:]
  predicted_values = []

  for _ in range(future_years):
    x_input = last_sequence.reshape((1, time_steps, 1))
    yhat = model.predict(x_input, verbose=0)
    predicted_values.append(yhat[0][0])
    last_sequence = np.append(last_sequence[1:], yhat[0])

  predicted_values = scaler.inverse_transform(np.array(predicted_values).reshape(-1, 1))

  return predicted_values, rmse, mae, r2

# Load the dataset
df = pd.read_csv('All.csv')
df['system:time_start'] = pd.to_datetime(df['system:time_start'])
df.set_index('system:time_start', inplace=True)

# Forecast for each state
for state in df['state_name'].unique():
    state_data = df[df['state_name'] == state]
    forecasts, rmse, mae, r2 = forecast_urban_sol(state_data)

    print(f"\nForecasts for {state}:")
    for i, forecast in enumerate(forecasts):
        print(f"Year {2025 + i}: {forecast[0]}")

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R² Score: {r2}")

    # You can add plotting and other analysis here for each state

<ipython-input-10-8da500a0e65f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)



Forecasts for Andhra Pradesh:
Year 2025: 70969.140625
Year 2026: 72823.046875
Year 2027: 74475.984375
Year 2028: 76168.0703125
Year 2029: 80110.5
RMSE: 6837.971875000003
MAE: 6837.971875000003
R² Score: nan


<ipython-input-10-8da500a0e65f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)



Forecasts for Maharashtra:
Year 2025: 354649.9375
Year 2026: 378159.40625
Year 2027: 397989.9375
Year 2028: 415434.625
Year 2029: 448779.6875
RMSE: 28111.161250000005
MAE: 28111.161250000005
R² Score: nan


<ipython-input-10-8da500a0e65f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def forecast_sol(state_data, time_steps=5, epochs=100, batch_size=32):
  """
  Forecasts Urban_SOL and Agriculture_SOL for a given state using LSTM.

  Args:
    state_data: DataFrame containing data for the specific state.
    time_steps: Number of time steps for input sequence.
    epochs: Number of epochs for model training.
    batch_size: Batch size for model training.

  Returns:
    A dictionary containing forecasted values for 2025 and 2026 for both
    Urban_SOL and Agriculture_SOL.
  """

  # Preprocess data
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')

  # Select relevant columns
  data = state_data[['Urban_SOL', 'Agriculture_SOL']].values

  # Normalize data
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(data)

  # Create dataset for LSTM
  X, Y = [], []
  for i in range(len(scaled_data) - time_steps - 1):
    a = scaled_data[i:(i + time_steps), :]  # Use all features as input
    X.append(a)
    Y.append(scaled_data[i + time_steps, :])  # Use both Urban_SOL and Agriculture_SOL as targets
  X, Y = np.array(X), np.array(Y)

  # Split into train and test sets
  train_size = int(len(X) * 0.8)
  X_train, X_test = X[:train_size], X[train_size:]
  Y_train, Y_test = Y[:train_size], Y[train_size:]

  # Build the LSTM model
  model = Sequential([
      LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
      Dropout(0.2),
      LSTM(units=50),
      Dropout(0.2),
      Dense(units=2)  # Output layer for both Urban_SOL and Agriculture_SOL
  ])

  model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

  # Forecast for 2025 and 2026
  last_sequence = scaled_data[-time_steps:]
  forecasts = []

  for _ in range(2):  # Forecast for 2 years (2025 and 2026)
    x_input = last_sequence.reshape((1, time_steps, 2))  # Use all features as input
    yhat = model.predict(x_input, verbose=0)
    forecasts.append(scaler.inverse_transform(yhat)[0])
    last_sequence = np.append(last_sequence[1:], yhat, axis=0)

  # Extract forecasted values
  forecasted_urban_sol_2025 = forecasts[0][0]
  forecasted_agriculture_sol_2025 = forecasts[0][1]
  forecasted_urban_sol_2026 = forecasts[1][0]
  forecasted_agriculture_sol_2026 = forecasts[1][1]

  return {
      "2025": {
          "Urban_SOL": forecasted_urban_sol_2025,
          "Agriculture_SOL": forecasted_agriculture_sol_2025
      },
      "2026": {
          "Urban_SOL": forecasted_urban_sol_2026,
          "Agriculture_SOL": forecasted_agriculture_sol_2026
      }
  }

# Load the dataset
df = pd.read_csv('All.csv')
df['system:time_start'] = pd.to_datetime(df['system:time_start'])
df.set_index('system:time_start', inplace=True)

# Forecast for each state
for state in df['state_name'].unique():
    state_data = df[df['state_name'] == state]
    forecasts = forecast_sol(state_data)

    print(f"\nForecasts for {state}:")
    print(f"2025: Urban_SOL = {forecasts['2025']['Urban_SOL']:.2f}, Agriculture_SOL = {forecasts['2025']['Agriculture_SOL']:.2f}")
    print(f"2026: Urban_SOL = {forecasts['2026']['Urban_SOL']:.2f}, Agriculture_SOL = {forecasts['2026']['Agriculture_SOL']:.2f}")

<ipython-input-11-62e0c0b04c80>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-11-62e0c0b04c80>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg


Forecasts for Andhra Pradesh:
2025: Urban_SOL = 71750.96, Agriculture_SOL = 592079.69
2026: Urban_SOL = 73671.77, Agriculture_SOL = 624424.56


<ipython-input-11-62e0c0b04c80>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-11-62e0c0b04c80>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg


Forecasts for Arunachal Pradesh:
2025: Urban_SOL = 527.63, Agriculture_SOL = 30556.65
2026: Urban_SOL = 563.37, Agriculture_SOL = 33417.93


<ipython-input-11-62e0c0b04c80>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-11-62e0c0b04c80>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

...


Forecasts for West Bengal:
2025: Urban_SOL = 152147.75, Agriculture_SOL = 437027.47
2026: Urban_SOL = 156548.39, Agriculture_SOL = 470570.72


<ipython-input-11-62e0c0b04c80>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-11-62e0c0b04c80>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg


Forecasts for Andaman and Nicobar Islands:
2025: Urban_SOL = 765.81, Agriculture_SOL = 832.37
2026: Urban_SOL = 832.86, Agriculture_SOL = 846.35


<ipython-input-11-62e0c0b04c80>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-11-62e0c0b04c80>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

KeyboardInterrupt: 

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from colorama import Fore, Style

def forecast_sol(state_data, time_steps=5, epochs=100, batch_size=32):
  """
  Forecasts Urban_SOL and Agriculture_SOL for a given state using LSTM.

  Args:
    state_data: DataFrame containing data for the specific state.
    time_steps: Number of time steps for input sequence.
    epochs: Number of epochs for model training.
    batch_size: Batch size for model training.

  Returns:
    A dictionary containing forecasted values for 2025 and 2026 for both
    Urban_SOL and Agriculture_SOL.
  """

  # Preprocess data
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')

  # Select relevant columns
  data = state_data[['Urban_SOL', 'Agriculture_SOL']].values

  # Normalize data
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(data)

  # Create dataset for LSTM
  X, Y = [], []
  for i in range(len(scaled_data) - time_steps - 1):
    a = scaled_data[i:(i + time_steps), :]  # Use all features as input
    X.append(a)
    Y.append(scaled_data[i + time_steps, :])  # Use both Urban_SOL and Agriculture_SOL as targets
  X, Y = np.array(X), np.array(Y)

  # Split into train and test sets
  train_size = int(len(X) * 0.8)
  X_train, X_test = X[:train_size], X[train_size:]
  Y_train, Y_test = Y[:train_size], Y[train_size:]

  # Build the LSTM model
  model = Sequential([
      LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
      Dropout(0.2),
      LSTM(units=50),
      Dropout(0.2),
      Dense(units=2)  # Output layer for both Urban_SOL and Agriculture_SOL
  ])

  model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

  # Forecast for 2025 and 2026
  last_sequence = scaled_data[-time_steps:]
  forecasts = []

  for _ in range(2):  # Forecast for 2 years (2025 and 2026)
    x_input = last_sequence.reshape((1, time_steps, 2))  # Use all features as input
    yhat = model.predict(x_input, verbose=0)
    forecasts.append(scaler.inverse_transform(yhat)[0])
    last_sequence = np.append(last_sequence[1:], yhat, axis=0)

  # Extract forecasted values
  forecasted_urban_sol_2025 = forecasts[0][0]
  forecasted_agriculture_sol_2025 = forecasts[0][1]
  forecasted_urban_sol_2026 = forecasts[1][0]
  forecasted_agriculture_sol_2026 = forecasts[1][1]

  return {
      "2025": {
          "Urban_SOL": forecasted_urban_sol_2025,
          "Agriculture_SOL": forecasted_agriculture_sol_2025
      },
      "2026": {
          "Urban_SOL": forecasted_urban_sol_2026,
          "Agriculture_SOL": forecasted_agriculture_sol_2026
      }
  }

# Load the dataset
df = pd.read_csv('All.csv')
df['system:time_start'] = pd.to_datetime(df['system:time_start'])
df.set_index('system:time_start', inplace=True)

# Get user input for the state
target_state = input("Enter the state name: ")

# Filter data for the specified state
state_data = df[df['state_name'] == target_state]

if not state_data.empty:
    forecasts = forecast_sol(state_data)

    print(f"\n{Fore.GREEN}Forecasts for {target_state}:{Style.RESET_ALL}")
    print(f"2025: Urban_SOL = {Fore.GREEN}{forecasts['2025']['Urban_SOL']:.2f}{Style.RESET_ALL}, Agriculture_SOL = {Fore.GREEN}{forecasts['2025']['Agriculture_SOL']:.2f}{Style.RESET_ALL}")
    print(f"2026: Urban_SOL = {Fore.GREEN}{forecasts['2026']['Urban_SOL']:.2f}{Style.RESET_ALL}, Agriculture_SOL = {Fore.GREEN}{forecasts['2026']['Agriculture_SOL']:.2f}{Style.RESET_ALL}")
else:
    print(f"{Fore.RED}No data found for state: {target_state}{Style.RESET_ALL}")

Enter the state name: Bihar


<ipython-input-26-181d6afd80a5>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Urban_SOL'] = pd.to_numeric(state_data['Urban_SOL'].str.replace(',', ''), errors='coerce')
<ipython-input-26-181d6afd80a5>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['Agriculture_SOL'] = pd.to_numeric(state_data['Agriculture_SOL'].str.replace(',', ''), errors='coerce')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg


Forecasts for Bihar:
2025: Urban_SOL = 79863.13, Agriculture_SOL = 958676.00
2026: Urban_SOL = 86625.16, Agriculture_SOL = 1066640.50
